# Train text model

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
cards = pd.read_csv('./AllPrintingsCSVFiles/cards.csv')
cards.columns

/var/folders/2s/c36gzps966l7g26p24rbth540000gn/T/ipykernel_943/4007862019.py:1: DtypeWarning: Columns (2,3,7,9,12,16,20,23,25,26,31,32,33,35,37,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cards = pd.read_csv('./AllPrintingsCSVFiles/cards.csv')


Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [3]:
cards = cards[['asciiName', 'availability',
       'colorIdentity',
       'colors', 'defense', 'flavorText',
       'frameEffects', 'keywords',
       'language', 'layout', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 
       'power', 'rarity', 
       'subtypes', 'supertypes', 'text','setCode',
       'toughness', 'type', 'types', 'uuid', 'variations',]]

In [ ]:
cards = cards[cards['language'] == 'English']
cards = cards[cards['setCode'].isin(['7ED', '8ED', '9ED', '10E'])]
cards = cards[~cards['type'].str.contains('Basic Land')]

In [8]:
cards.columns

Index(['asciiName', 'availability', 'colorIdentity', 'colors', 'defense',
       'flavorText', 'frameEffects', 'keywords', 'language', 'layout', 'life',
       'loyalty', 'manaCost', 'manaValue', 'name', 'power', 'rarity',
       'subtypes', 'supertypes', 'text', 'setCode', 'toughness', 'type',
       'types', 'uuid', 'variations'],
      dtype='object')

In [10]:
#cards_name_text = cards[['name', 'type', 'text']]
cards_info = cards[['name', 'manaCost', 'type', 'text', 'power', 'toughness', 'flavorText']]

In [11]:
cards_info

,name,manaCost,type,text,power,toughness,flavorText
0,Ancestor's Chosen,{5}{W}{W},Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,NaN
1,Ancestor's Chosen,{5}{W}{W},Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,"""The will of all, by my hand done."""
2,Angel of Mercy,{4}{W},Creature — Angel,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,3,Every tear shed is a drop of immortality.
3,Angel of Mercy,{4}{W},Creature — Angel,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,3,Every tear shed is a drop of immortality.
4,Angelic Blessing,{2}{W},Sorcery,Target creature gets +3/+3 and gains flying un...,NaN,NaN,Only the warrior who can admit mortal weakness...
...,...,...,...,...,...,...,...
7110,Urza's Power Plant,NaN,Land — Urza's Power-Plant,{T}: Add {C}. If you control an Urza's Mine an...,NaN,NaN,NaN
7111,Urza's Tower,NaN,Land — Urza's Tower,{T}: Add {C}. If you control an Urza's Mine an...,NaN,NaN,NaN
7112,Urza's Tower,NaN,Land — Urza's Tower,{T}: Add {C}. If you control an Urza's Mine an...,NaN,NaN,NaN
7113,Yavimaya Coast,NaN,Land,{T}: Add {C}.\n{T}: Add {G} or {U}. Yavimaya C...,NaN,NaN,NaN


In [12]:
# Treat Nans
cards_info.loc[:, 'manaCost'] = cards_info['manaCost'].fillna('{Z}')
cards_info.loc[:, 'text'] = cards_info['text'].fillna('---')
cards_info.loc[:, 'power'] = cards_info['power'].fillna('-')
cards_info.loc[:, 'toughness'] = cards_info['toughness'].fillna('-')
cards_info.loc[:, 'flavorText'] = cards_info['flavorText'].fillna('---')

In [13]:
text_lengths = cards_info['text'].astype(str).apply(len)
length_counter = Counter(text_lengths)
print(max(length_counter))

370


# Text Generator

In [ ]:
# Define a special token for separation
separator_token = "[SEP]"
endoftext_token = "<|endoftext|>"

# Add the special token to the tokenizer

# Connect the name and text fields with the special token

cards_info['info'] = cards_info.apply(lambda row: f"{row['name']} {separator_token} {row['manaCost']} {separator_token} {row['type']} {separator_token} {row['text']} {separator_token} {row['flavorText']} {separator_token} {row['power']}/{row['toughness']} {endoftext_token}", axis=1)


# Display the updated DataFrame
cards_info.head()

/var/folders/2s/c36gzps966l7g26p24rbth540000gn/T/ipykernel_943/2003439509.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cards_info['info'] = cards_info.apply(lambda row: f"{row['name']} {separator_token} {row['manaCost']} {separator_token} {row['type']} {separator_token} {row['text']} {separator_token} {row['flavorText']} {separator_token} {row['power']}/{row['toughness']} {endoftext_token}", axis=1)


,name,manaCost,type,text,power,toughness,flavorText,info
0,Ancestor's Chosen,{5}{W}{W},Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,---,Ancestor's Chosen [SEP] {5}{W}{W} [SEP] Creatu...
1,Ancestor's Chosen,{5}{W}{W},Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,"""The will of all, by my hand done.""",Ancestor's Chosen [SEP] {5}{W}{W} [SEP] Creatu...
2,Angel of Mercy,{4}{W},Creature — Angel,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,3,Every tear shed is a drop of immortality.,Angel of Mercy [SEP] {4}{W} [SEP] Creature — A...
3,Angel of Mercy,{4}{W},Creature — Angel,"Flying\nWhen Angel of Mercy enters, you gain 3...",3,3,Every tear shed is a drop of immortality.,Angel of Mercy [SEP] {4}{W} [SEP] Creature — A...
4,Angelic Blessing,{2}{W},Sorcery,Target creature gets +3/+3 and gains flying un...,-,-,Only the warrior who can admit mortal weakness...,Angelic Blessing [SEP] {2}{W} [SEP] Sorcery [S...


In [15]:
len(cards_info['info'][0])

272

In [16]:
# drop all rows that have name_text > 512
cards_info = cards_info[cards_info['info'].apply(len) <= 512]

In [17]:
from datasets import Dataset
dataset = Dataset.from_pandas(cards_info[['info']])

In [18]:
from transformers import (
    AutoTokenizer,
)
MODEL_NAME = 'openai-community/gpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token



def tokenize_function(examples):
    outputs = tokenizer(
        examples["info"], truncation=True, padding="max_length", max_length=512
    )
    outputs["labels"] = outputs["input_ids"].copy()  # Set labels for loss calculation
    return outputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2486 [00:00<?, ? examples/s]

In [ ]:

from transformers import AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = 'openai-community/gpt2'

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

model.resize_token_embeddings(len(tokenizer))

model = model.to("mps")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Turn off Masked Language Modeling; use CLM
)

In [ ]:
from datasets import DatasetDict

# Split the tokenized dataset into train and evaluation sets
split_datasets = tokenized_datasets.train_test_split(test_size=0.1)

# Access train and eval datasets
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Optional: Convert to a DatasetDict for Hugging Face workflows
dataset_dict = DatasetDict({"train": train_dataset, "eval": eval_dataset})


In [22]:
# Ensure data collator and dataset are compatible
import torch
device = 'mps'
def custom_data_collator(features):
    # Convert tensors to the MPS device, but don't change their dtype
    return {
        key: torch.tensor([f[key] for f in features], dtype=torch.long if key == "input_ids" else torch.float).to(device)
        for key in features[0]
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=40,
    per_device_train_batch_size=8,
    #per_device_eval_batch_size=4,
    save_steps=100,
    save_total_limit=4,
    logging_dir="./logs",
    logging_steps=100,  # Log metrics every 10 steps
    learning_rate=5e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-Tune the Model
trainer.train()

/var/folders/2s/c36gzps966l7g26p24rbth540000gn/T/ipykernel_943/2620514094.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/11200 [00:00<?, ?it/s]

{'loss': 1.9098, 'grad_norm': 6.738430500030518, 'learning_rate': 4.955357142857143e-05, 'epoch': 0.36}
{'loss': 1.5136, 'grad_norm': 6.070378303527832, 'learning_rate': 4.910714285714286e-05, 'epoch': 0.71}
{'loss': 1.3249, 'grad_norm': 5.582805633544922, 'learning_rate': 4.866071428571429e-05, 'epoch': 1.07}
{'loss': 1.1124, 'grad_norm': 5.896852493286133, 'learning_rate': 4.8214285714285716e-05, 'epoch': 1.43}
{'loss': 1.0675, 'grad_norm': 5.8383378982543945, 'learning_rate': 4.7767857142857144e-05, 'epoch': 1.79}
{'loss': 0.9457, 'grad_norm': 5.409012317657471, 'learning_rate': 4.732142857142857e-05, 'epoch': 2.14}
{'loss': 0.8357, 'grad_norm': 4.424046993255615, 'learning_rate': 4.6875e-05, 'epoch': 2.5}
{'loss': 0.7916, 'grad_norm': 5.051490306854248, 'learning_rate': 4.642857142857143e-05, 'epoch': 2.86}
{'loss': 0.6871, 'grad_norm': 4.665743827819824, 'learning_rate': 4.598214285714286e-05, 'epoch': 3.21}
{'loss': 0.6081, 'grad_norm': 3.9982893466949463, 'learning_rate': 4.5535

TrainOutput(global_step=11200, training_loss=0.1955780295389039, metrics={'train_runtime': 9686.1901, 'train_samples_per_second': 9.238, 'train_steps_per_second': 1.156, 'total_flos': 2.338041102336e+16, 'train_loss': 0.1955780295389039, 'epoch': 40.0})

In [60]:
# Function to truncate a string at the first occurrence of the pattern
def truncate_at_pattern(dic, pattern):
    text = dic['generated_text']
    match = re.search(pattern, text)
    if match:
        # Get the end position of the match and truncate the string
        return text[:match.end()]
    return text  # Return the original text if no match is found


In [ ]:
from transformers import pipeline
from transformers import AutoModelForCausalLM
checkpoint2 = "./results/checkpoint-11200"
model1 = AutoModelForCausalLM.from_pretrained(checkpoint2)
generator = pipeline('text-generation', model=model1, tokenizer=tokenizer)
pattern = r"\[SEP\] [-*\d]/[-*\d]"
outputs = generator("Voracious Demon", max_length=512, num_return_sequences=5, pad_token_id=tokenizer.eos_token_id)
#print(outputs)
# # Print the outputs
# # Apply truncation
truncated_strings = [truncate_at_pattern(s, pattern) for s in outputs]

# Print the results
for i, result in enumerate(truncated_strings):
    print(result)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Voracious Demon [SEP] {4}{B}{B} [SEP] Creature — Phyrexian Demon [SEP] At the beginning of your upkeep, Voracious Demon deals 5 damage to target creature. [SEP] --- [SEP] 6/6
Voracious Demon [SEP] {4}{B}{B} [SEP] Creature — Phyrexian Vampire Noble [SEP] Flying (This creature can't be blocked except by creatures with flying or reach.)\nWhenever Voracious Demon deals combat damage to a player, that player exiles the top four cards of their library. If two or more of those cards have the same name, repeat this process. [SEP] --- [SEP] 4/4
Voracious Demon [SEP] {2}{B}{B} [SEP] Creature — Phyrexian Demon [SEP] When Voracious Demon enters, target opponent sacrifices a creature of their choice. [SEP] He's hungry for the taste of death. And furious for the violence it elicits. [SEP] 2/2
Voracious Demon [SEP] {4}{B}{B} [SEP] Creature — Phyrexian Demon [SEP] At the beginning of your upkeep, you may Virous Demon deal 3 damage to target creature. [SEP] --- [SEP] 3/3
Voracious Demon [SEP] {2}{B}{B}